# MTG: Creature Power-Creep

## Data Cleaning

Converting the json file downloaded from scryfall to csv:

In [48]:
import pandas as pd

df = pd.read_json ("scryfall-default-cards.json", encoding = "utf-8")

The character '—' is causing problems, we will replace it:

In [49]:
df["type_line"] = df["type_line"].str.replace(' —', ':')

In [50]:
#df.to_csv("cards.csv", index = None)

Opening and quickly inspecting the csv

In [51]:
#df = pd.read_csv("cards.csv")
df.head()

,object,id,oracle_id,multiverse_ids,name,printed_name,lang,released_at,uri,scryfall_uri,...,watermark,color_indicator,loyalty,promo_types,mtgo_id,life_modifier,hand_modifier,arena_id,variation_of,mtgo_foil_id
0,card,02efd6cf-1ed0-44b3-b3b9-94a45696e1f3,09aeea91-b1dc-443f-a509-4758f052c0a7,[],Voltaic Key,通電式キー,ja,2020-03-26,https://api.scryfall.com/cards/02efd6cf-1ed0-4...,https://scryfall.com/card/pmei/35/ja/%E9%80%9A...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,94355044-e67a-4605-a37b-e879784fc9e5,9d3c7c96-056f-408e-a834-fa45a430d3d4,[],"Daxos, Blessed by the Sun",太陽に祝福されしダクソス,ja,2020-03-01,https://api.scryfall.com/cards/94355044-e67a-4...,https://scryfall.com/card/pmei/36/ja/%E5%A4%AA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,e8fb396f-3ae1-4705-bf94-3e1b0556e910,3c304eaa-09e9-4439-b35d-4b05a54c2fa7,[],Acorn Stash,NaN,en,2020-02-29,https://api.scryfall.com/cards/e8fb396f-3ae1-4...,https://scryfall.com/card/tund/6/acorn-stash?u...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,628c542b-7579-4070-9143-6f1f7221468f,d82528ac-88f7-4c38-9b1c-c33e6faea532,[],Giant Teddy Bear,NaN,en,2020-02-29,https://api.scryfall.com/cards/628c542b-7579-4...,https://scryfall.com/card/tund/5/giant-teddy-b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,6819b8c9-73a3-4e8e-ad7a-95cffabf873a,eaf318f0-2cad-437c-8162-78462ecbc235,[],Dragon,NaN,en,2020-02-29,https://api.scryfall.com/cards/6819b8c9-73a3-4...,https://scryfall.com/card/tund/4/dragon?utm_so...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.shape

(51230, 71)

In [53]:
df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'name', 'printed_name',
       'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       'image_uris', 'mana_cost', 'cmc', 'type_line', 'printed_type_line',
       'oracle_text', 'printed_text', 'colors', 'color_identity', 'legalities',
       'games', 'reserved', 'foil', 'nonfoil', 'oversized', 'promo', 'reprint',
       'variation', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri',
       'scryfall_set_uri', 'rulings_uri', 'prints_search_uri',
       'collector_number', 'digital', 'rarity', 'card_back_id', 'artist',
       'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art',
       'textless', 'booster', 'story_spotlight', 'edhrec_rank', 'related_uris',
       'power', 'toughness', 'frame_effects', 'all_parts', 'tcgplayer_id',
       'preview', 'flavor_text', 'card_faces', 'watermark', 'color_indicator',
       'loyalty', 'promo_types', 'mtgo_id', 'life_modifier', 'hand_modifier',
 

### Narrowing down to english cards

In [54]:
english_cards = df[df["lang"] == "en"]
english_cards.shape

(49223, 71)

### Narrowing down to core set cards

In [55]:
english_cards["set_type"].unique()

array(['token', 'funny', 'promo', 'expansion', 'memorabilia', 'box',
       'commander', 'core', 'spellbook', 'draft_innovation',
       'masterpiece', 'planechase', 'masters', 'treasure_chest',
       'starter', 'duel_deck', 'from_the_vault', 'archenemy',
       'premium_deck', 'vanguard'], dtype=object)

In [56]:
english_core_cards = english_cards[english_cards["set_type"] == "core"]
english_core_cards.shape

(6743, 71)

Checking that we indeed have the complete core sets and that all the cards we select make sense for our analysis

In [57]:
english_core_cards["set"].unique()

array(['m20', 'm19', 'ori', 'm15', 'm14', 'm13', 'm12', 'm11', 'm10',
       '10e', '9ed', '8ed', '7ed', '6ed', '5ed', '4ed', 'sum', '3ed',
       '2ed', 'leb', 'lea'], dtype=object)

In [58]:
m20 = english_core_cards[english_core_cards["set"] == "m20"]
m20.shape

(345, 71)

After checking on scryfall.com, we realise that, for the example of M20, we do have all the cards.
However, we also have some non-draftable cards, like the Buy-a-box promo or cards for planeswalker decks.
For our analysis, it would be best to remove these non-booster cards, as they tend to be different and will not make for a fair comparison with older sets not having these subsets.
We will now proceed to only keep in-booster cards:

In [59]:
english_core_cards["booster"].unique()

array([False,  True])

In [60]:
english_draftable_core_cards = english_core_cards[english_core_cards["booster"] == True]
english_draftable_core_cards[english_draftable_core_cards["set"] == "m20"].shape

(280, 71)

There is indeed 280 draftable cards in M20, our operation succeeded.

### Narrowing down to creatures

In [61]:
english_draftable_core_cards["type_line"].unique()

array(['Basic Land: Forest', 'Basic Land: Mountain', 'Basic Land: Swamp',
       'Basic Land: Island', 'Basic Land: Plains', 'Land', 'Artifact',
       'Artifact Creature: Golem', 'Artifact Creature: Construct',
       'Artifact Creature: Scarecrow', 'Artifact: Equipment',
       'Legendary Artifact Creature: Scout', 'Artifact Creature: Knight',
       'Artifact Creature: Bird', 'Legendary Creature: Elemental Horror',
       'Creature: Zombie Wizard', 'Creature: Human Knight',
       'Creature: Elemental', 'Legendary Creature: Elemental',
       'Creature: Ogre Berserker', 'Enchantment',
       'Creature: Elemental Wizard', 'Legendary Creature: Bird Wizard',
       'Legendary Creature: Elf Advisor',
       'Legendary Creature: Human Cleric', 'Creature: Treefolk Soldier',
       'Creature: Bird Spirit', 'Creature: Zombie Knight',
       'Creature: Elf Scout', 'Enchantment: Aura',
       'Creature: Elemental Horror', 'Creature: Hydra',
       'Legendary Planeswalker: Vivien', 'Instant', 

We will only keep rows where the type_line contains "Creature"

In [62]:
english_draftable_core_creatures = english_draftable_core_cards[english_draftable_core_cards["type_line"].str.contains("Creature")]
english_draftable_core_creatures.shape

(2801, 71)

In [63]:
(english_draftable_core_creatures["set"] == "m20").sum()

141

There is indeed 141 draftable creatures in M20.

### Narrowing note
We started with 51 230 cards and we now only work with 2 801, which is about 5.47% of the starting rows.

### Excluding unnecessary columns

In [64]:
relevant_columns = ["id", "name", "mana_cost", "cmc", "type_line", "colors", "set", "set_name", "rarity", "power", "toughness", "released_at"]
# We only take the selected columns, copy the data, and reset the index without keeping the old one
selected_creatures = english_draftable_core_creatures[relevant_columns].copy().reset_index(drop = True)

In [65]:
selected_creatures.head()

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
0,1b4de70a-729b-4566-b6f3-c76f551405a5,Stone Golem,{5},5.0,Artifact Creature: Golem,[],m20,Core Set 2020,common,4,4,2019-07-12
1,6ed61426-e652-4b48-b936-8be9b6b57731,Steel Overseer,{2},2.0,Artifact Creature: Construct,[],m20,Core Set 2020,rare,1,1,2019-07-12
2,39f2d2e8-a9ed-42cc-9a8a-fc697a7251e5,Scuttlemutt,{3},3.0,Artifact Creature: Scarecrow,[],m20,Core Set 2020,uncommon,2,2,2019-07-12
3,8dd2ebbe-71c6-405b-bad0-5680f0ff575c,Salvager of Ruin,{3},3.0,Artifact Creature: Construct,[],m20,Core Set 2020,uncommon,2,1,2019-07-12
4,5c9fdb01-ba52-4510-897e-0d69558fdaee,Prismite,{2},2.0,Artifact Creature: Golem,[],m20,Core Set 2020,common,2,1,2019-07-12


In [66]:
# We will use 'sc' as a shorthand to make lines shorter
sc = selected_creatures

### Cleaning column data

In [67]:
sc.dtypes

id                     object
name                   object
mana_cost              object
cmc                   float64
type_line              object
colors                 object
set                    object
set_name               object
rarity                 object
power                  object
toughness              object
released_at    datetime64[ns]
dtype: object

We would like to convert to int the cmc, power and toughness columns.
We will proceed to examine their unique values to see if we can.

#### 'cmc' column

In [68]:
sc["cmc"].unique()

array([ 5.,  2.,  3.,  4.,  7.,  6.,  1.,  8.,  0.,  9., 11.])

In [69]:
sc["cmc"] = sc["cmc"].astype(int)

#### 'power' and 'toughness' columns

In [70]:
sc["power"].unique()

array(['4', '1', '2', '3', '*', '5', '7', '0', '8', '6', nan, '10', '9',
       '11', '2+*'], dtype=object)

In [71]:
sc["toughness"].unique()

array(['4', '1', '2', '3', '5', '7', '6', '8', '0', nan, '10', '*', '9',
       '11', '1+*', '7-*', '2+*'], dtype=object)

We note that there are non-numeric values, '*', '+' and 'nan'. The first two indicate a dynamic value. The 'nan' is concerning.
We will now inspect them closely to see if we can approximate their value to a int, or need to drop them:

In [72]:
# creatures that have '*' in their power and/or toughness
peculiar_creatures = sc[(sc["power"].str.contains('\*')) | (sc["toughness"].str.contains('\*'))]
peculiar_creatures

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
20,edec85ce-7daa-48c2-b25d-b22941e01e73,Ironroot Warlord,{1}{G}{W},3,Creature: Treefolk Soldier,"[G, W]",m20,Core Set 2020,uncommon,*,5,2019-07-12
156,6e0def77-3528-40fb-a6b2-c3d1e31ade65,Enigma Drake,{1}{U}{R},3,Creature: Drake,"[R, U]",m19,Core Set 2019,uncommon,*,4,2018-07-13
292,eb12b1d8-c53e-4d48-89e5-2168ff34a853,Zendikar Incarnate,{2}{R}{G},4,Creature: Elemental,"[G, R]",ori,Magic Origins,uncommon,*,4,2015-07-17
354,c7c6dbe5-3375-42c2-88f5-8ead0dc2b094,Revenant,{4}{B},5,Creature: Spirit,[B],ori,Magic Origins,uncommon,*,*,2015-07-17
446,dabe0865-5420-463e-9138-ccd805be8b31,Kalonian Twingrove,{5}{G},6,Creature: Treefolk Warrior,[G],m15,Magic 2015,rare,*,*,2014-07-18
...,...,...,...,...,...,...,...,...,...,...,...,...
2674,fc78dced-27d2-441a-b63b-32356bc33747,Nightmare,{5}{B},6,Creature: Nightmare Horse,[B],leb,Limited Edition Beta,rare,*,*,1993-10-04
2727,e2b15221-c8b0-4861-9f8b-8a65834ad499,Gaea's Liege,{3}{G}{G}{G},6,Creature: Avatar,[G],lea,Limited Edition Alpha,rare,*,*,1993-08-05
2745,8fe3fd83-969c-4add-888f-86f4306b067c,Keldon Warlord,{2}{R}{R},4,Creature: Human Barbarian,[R],lea,Limited Edition Alpha,uncommon,*,*,1993-08-05
2765,b3724e40-0622-4aee-9334-6c9fff88bcd5,Plague Rats,{2}{B},3,Creature: Rat,[B],lea,Limited Edition Alpha,common,*,*,1993-08-05


In [73]:
peculiar_creatures["set"].value_counts()

5ed    8
4ed    7
10e    7
7ed    6
9ed    6
8ed    5
3ed    5
sum    5
leb    4
lea    4
6ed    4
2ed    4
m13    3
m15    2
ori    2
m12    2
m10    2
m20    1
m19    1
m14    1
Name: set, dtype: int64

There are 79 creatures that have a dynamic power and/or toughness. There are too many to estimate one by one, we will drop them. They are not concentrated on a few sets, so it will not impact the analysis in a significant way.
It represents 79/2801 = 2.82% of the rows.
We drop them:

In [74]:
sc.drop(peculiar_creatures.index, inplace = True)
sc.shape

(2722, 12)

In [75]:
# creatures that have a np.nan power and/or toughness
nan_stat_creatures = sc[(sc["power"].isnull()) | (sc["toughness"].isnull())]
nan_stat_creatures

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
155,7b215968-93a6-4278-ac61-4e3e8c3c3943,"Nicol Bolas, the Ravager // Nicol Bolas, the A...",NaN,4,Legendary Creature: Elder Dragon // Legendary ...,NaN,m19,Core Set 2019,mythic,NaN,NaN,2018-07-13
315,ff0063da-ab6b-499d-8e87-8b34d46f0372,"Nissa, Vastwood Seer // Nissa, Sage Animist",NaN,3,Legendary Creature: Elf Scout // Legendary Pla...,NaN,ori,Magic Origins,mythic,NaN,NaN,2015-07-17
344,b0d6caf0-4fa8-4ec5-b7f4-1307474d1b13,"Chandra, Fire of Kaladesh // Chandra, Roaring ...",NaN,3,Legendary Creature: Human Shaman // Legendary ...,NaN,ori,Magic Origins,mythic,NaN,NaN,2015-07-17
360,9f25e1cf-eeb4-458d-8fb2-b3a2f86bdd54,"Liliana, Heretical Healer // Liliana, Defiant ...",NaN,3,Legendary Creature: Human Cleric // Legendary ...,NaN,ori,Magic Origins,mythic,NaN,NaN,2015-07-17
388,02d6d693-f1f3-4317-bcc0-c21fa8490d38,"Jace, Vryn's Prodigy // Jace, Telepath Unbound",NaN,2,Legendary Creature: Human Wizard // Legendary ...,NaN,ori,Magic Origins,mythic,NaN,NaN,2015-07-17
405,58c39df6-b237-40d1-bdcb-2fe5d05392a9,"Kytheon, Hero of Akros // Gideon, Battle-Forged",NaN,1,Legendary Creature: Human Soldier // Legendary...,NaN,ori,Magic Origins,mythic,NaN,NaN,2015-07-17


These 6 creatures are double-faced cards, with one side being a creature, and the other a planeswalker.

In [76]:
# we like in the non-narrowed df to see if we can find the power and toughness for these creatures
df[df["name"].str.contains("Nicol Bolas, the Ravager")][relevant_columns]

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
7035,6ab66344-4959-4c7b-8828-b1a89b274e0b,"Nicol Bolas, the Ravager // Nicol Bolas, the A...",NaN,4.0,Legendary Creature: Elder Dragon // Legendary ...,NaN,pm19,Core Set 2019 Promos,mythic,NaN,NaN,2018-07-13
7233,7b215968-93a6-4278-ac61-4e3e8c3c3943,"Nicol Bolas, the Ravager // Nicol Bolas, the A...",NaN,4.0,Legendary Creature: Elder Dragon // Legendary ...,NaN,m19,Core Set 2019,mythic,NaN,NaN,2018-07-13


It looks like that the complete list of cards contains 2 cards for each double-faced cards, but none have power or touhness values.
We will drop these rows:

In [77]:
sc.drop(nan_stat_creatures.index, inplace = True)
sc.shape

(2716, 12)

Now we can convert the 'power' and 'toughness' columns:

In [78]:
sc["power"] = sc["power"].astype(int)
sc["toughness"] = sc["toughness"].astype(int)
sc.dtypes

id                     object
name                   object
mana_cost              object
cmc                     int32
type_line              object
colors                 object
set                    object
set_name               object
rarity                 object
power                   int32
toughness               int32
released_at    datetime64[ns]
dtype: object

### Checking for other NaN values

In [79]:
sc.isnull().sum()

id             0
name           0
mana_cost      0
cmc            0
type_line      0
colors         0
set            0
set_name       0
rarity         0
power          0
toughness      0
released_at    0
dtype: int64

There are no NaN values left.

### Checking for outliers or unexploitable values

In [80]:
# power value distribution in %
sc["power"].value_counts(normalize = True).sort_index() * 100

0      7.695140
1     25.478645
2     32.069219
3     13.991163
4      9.020619
5      5.559647
6      3.534610
7      1.472754
8      0.773196
9      0.220913
10     0.110457
11     0.073638
Name: power, dtype: float64

In [81]:
# toughness value distribution in %
sc["toughness"].value_counts(normalize = True).sort_index() * 100

0      1.325479
1     27.172312
2     25.147275
3     18.372607
4     12.665685
5      7.511046
6      4.344624
7      1.951399
8      1.030928
9      0.294551
10     0.110457
11     0.073638
Name: toughness, dtype: float64

We note that 1.33% of creatures have a toughness of 0, which we should investigate (as a creature with 0 power dies instantly)

In [87]:
no_toughness = sc[sc["toughness"] == 0]
no_toughness.shape

(36, 12)

In [88]:
no_toughness.head()

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
87,77972745-8689-4733-9a9d-39ae9d6273a2,Embodiment of Agonies,{1}{B}{B},3,Creature: Demon,[B],m20,Core Set 2020,rare,0,0,2019-07-12
171,84127b83-e75a-4f12-92ca-46f50bb89699,Hungering Hydra,{X}{G},1,Creature: Hydra,[G],m19,Core Set 2019,rare,0,0,2018-07-13
248,5b3ffc69-f21b-410e-8993-8c1b4669fc19,Mirror Image,{2}{U},3,Creature: Shapeshifter,[U],m19,Core Set 2019,uncommon,0,0,2018-07-13
286,791c21fb-fc78-4106-9a42-abc73f41ab8b,Hangarback Walker,{X}{X},0,Artifact Creature: Construct,[],ori,Magic Origins,rare,0,0,2015-07-17
433,4f8dc77e-f003-4c25-8394-cda22e3ea039,Undergrowth Scavenger,{3}{G},4,Creature: Fungus Horror,[G],m15,Magic 2015,common,0,0,2014-07-18


These creatures all have a variable power and/or toughness. We can't exploit them for our analysis, so we will drop them. They represent only 1.33% of our rows.

In [89]:
sc.drop(no_toughness.index, inplace = True)
sc.shape

(2680, 12)

In [90]:
no_power = sc[sc["power"] == 0]
no_power.shape

(173, 12)

In [92]:
no_power["type_line"].unique()

array(['Creature: Hydra', 'Creature: Elemental Druid', 'Creature: Wolf',
       'Creature: Ox', 'Artifact Creature: Wall', 'Creature: Plant Wall',
       'Creature: Dragon Egg', 'Creature: Vampire', 'Creature: Wall',
       'Creature: Horse Fish', 'Artifact Creature: Golem',
       'Creature: Starfish', 'Creature: Illusion Wall',
       'Artifact Creature: Thopter', 'Creature: Insect',
       'Creature: Zombie Wall', 'Creature: Dragon',
       'Artifact Creature: Construct', 'Creature: Skeleton',
       'Creature: Kraken', 'Creature: Human Soldier', 'Creature: Bird',
       'Creature: Bird Egg', 'Creature: Cat Monk',
       'Creature: Human Druid', 'Creature: Elemental',
       'Creature: Human Cleric', 'Creature: Bat', 'Creature: Plant',
       'Creature: Spirit', 'Creature: Human Cleric Shaman',
       'Artifact Creature: Horse', 'Artifact Creature: Beast',
       'Creature: Wombat', 'Creature: Shade', 'Creature: Merfolk',
       'Artifact Creature: Bird', 'Creature: Human Wizard'], 

In [94]:
no_power.head()

,id,name,mana_cost,cmc,type_line,colors,set,set_name,rarity,power,toughness,released_at
2734,140e567c-6e4a-42b0-8084-d6c9695ae802,Wall of Stone,{1}{R}{R},3,Creature: Wall,[R],lea,Limited Edition Alpha,uncommon,0,8,1993-08-05
2735,efcf12cd-fb70-444e-9641-73ffa0e8f16e,Wall of Fire,{1}{R}{R},3,Creature: Wall,[R],lea,Limited Edition Alpha,uncommon,0,5,1993-08-05
2759,a1a6f8e9-7bc1-4151-b55f-acf877b1a7a6,Will-o'-the-Wisp,{B},1,Creature: Spirit,[B],lea,Limited Edition Alpha,rare,0,1,1993-08-05
2771,d0bd76c8-4cff-4c15-9686-7a299b589814,Frozen Shade,{2}{B},3,Creature: Shade,[B],lea,Limited Edition Alpha,common,0,1,1993-08-05
2777,41faed1a-ded8-49ee-8e2a-c60d377775d7,Wall of Water,{1}{U}{U},3,Creature: Wall,[U],lea,Limited Edition Alpha,uncommon,0,5,1993-08-05


We note that creatures with a power of 0 fall into 3 categories:
* creatures with variable stats (power & toughness) that are defined with a 'X' in the mana cost (unexploitable)
* creatures with a legitimate power of 0, most of them being walls (exploitable)

We will then proceed to drop any creature with a 'X' in the mana cost, as they are not exploitable.
By doing this, we will keep some creatures that have stats defined by something else in their rules text, but it would be too complicated to narrow them.

In [95]:
creatures_with_x = sc[sc["mana_cost"].str.contains('X')]
creatures_with_x.shape

(1, 12)

In [97]:
sc.drop(creatures_with_x.index, inplace = True)
sc.shape

(2679, 12)

## Creating a complete dataframe for our analysis
For our analysis, we would like to build a dataframe containing everything we need.
Each row will be set, and for each set we will have the following informations:
* code: set code
* name: set full name
* release: release date
* display_str: a string containing the set code and date for our plots
* avg_cmc: avg cmc of creatures in this set
* avg_power: avg power of creatures in this set
* avg_toughness: avg toughness of creatures in this set
* avg_effectiveness: positive or negative percentage translating how much better or worse the stats of an average creature of this set is compared to what is expected. The expected stats of a creature is calculated as the average stats of a creature with this exact mana cost in any core set.

In order to build thid dataframe, we will first need to add some columns to our 'sc' dataframe:
* stats: sum of the power & toughness
* expected_stats: expected stats for a creature of this exact mana cost (calculated from the average in core sets)
* effectiveness: positive or negative percentage translating how much better or worse the stats of this creature are compare to it's expected stats

### Adding the "stats"column
It is simply the sum of the power & toughness

In [99]:
sc["stats"] = sc["power"] + sc["toughness"]
sc["stats"].value_counts(normalize = True).sort_index() * 100

1      1.754386
2     19.335573
3      8.771930
4     23.889511
5      9.779769
6     11.944756
7      4.441956
8      7.390817
9      2.351624
10     4.180664
11     1.194476
12     2.239642
13     0.298619
14     1.306458
15     0.074655
16     0.634565
18     0.223964
20     0.111982
22     0.074655
Name: stats, dtype: float64

We note that there are significantly more cards with even stats than odd stats. That is probably due to the fact that square stats (power and toughness are equal) are easier to process, so are prefered to unsquare stats.

In [103]:
square_preference_factor = (sc["stats"] % 2 == 0).sum() / (sc["stats"] % 2 == 1).sum()
square_preference_factor

2.48828125

There are about 2.5 times more creatures with even stats than odd stats!

### Preparing for the "expected_stats" column
Expected stats for a creature of this exact mana cost (calculated from the average in core sets).
To have a good model, for each creature, I will define it's cost at being the expected stats value it should have, based on the average stats value of a creature having this exact mana-cost in the sc dataframe (exploitable core set creatures).
For this process to work well, we need several cases for each mana-cost.

In [114]:
sc["mana_cost"].value_counts().tail(40)

{5}{G}{G}{G}       2
{3}{U}{U}{U}       2
{1}{R}{W}          1
{4}{W}{U}{B}       1
{6}{U}{U}          1
{3}{B}{B}{B}{B}    1
{1}{W}{W}{W}       1
{1}{G}{U}{R}       1
{3}{G}{G}{G}{G}    1
{U}{R}             1
{1}{W}{U}          1
{W}{B}             1
{1}{W}{B}          1
{R}{W}             1
{3}{U}{B}          1
{3}{R}{G}{W}       1
{6}{B}{B}{B}       1
{1}{R}{G}          1
{3}{B}{R}{G}       1
{4}{U}{B}          1
{5}{B}{B}{B}       1
{1}{U}{R}{W}       1
{R}{G}             1
{6}{U}             1
{2}{B}{G}          1
{2}{B}{G}{U}       1
{W}{U}{B}{R}{G}    1
{W}{B}{G}          1
{R}{W}{B}          1
{2}{U}{R}          1
{1}{U}{B}          1
{3}{G}{G}{G}       1
{2}{G}{U}          1
{11}               1
{1}{B}{G}          1
{2}{U}{U}{U}       1
{1}{G}{W}{U}       1
{3}{W}{B}          1
{5}{B}{B}          1
{G}{G}{G}{G}{G}    1
Name: mana_cost, dtype: int64

We note that some mana costs are present on exactly one card. These cards are unexploitable, as we need at least two occurences per mana cost to observe an evolution. We will drop them.

In [132]:
def drop_singleton_mana_costs(df):
    #finds them
    singleton_mana_costs = []
    for mana_cost in df["mana_cost"].unique():
        occurrences = sc["mana_cost"].value_counts()[mana_cost] 
        if occurrences == 1:
            singleton_mana_costs.append(mana_cost)       
    #drops them
    for s_cost in singleton_mana_costs:
        lines_to_drop = df[df["mana_cost"] == s_cost]
        df.drop(lines_to_drop.index, inplace = True)
    
drop_singleton_mana_costs(sc)
sc.shape

(2641, 13)

We indeed dropped 38 rows.

### Adding the "expected_stats" column

In [141]:
def compute_expected_stats_for_cost(mana_cost, df):
    cards_having_this_cost = df[df["mana_cost"] == mana_cost]
    return cards_having_this_cost["stats"].mean()

def get_expected_stats_dictio(df):
    mana_costs = df["mana_cost"].unique()
    expected_stats_per_cost = {}
    for mana_cost in mana_costs:
        expected_stats_per_cost[mana_cost] = compute_expected_stats_for_cost(mana_cost, df)
    return expected_stats_per_cost

get_expected_stats_dictio(sc)

# TODO: create column

{'{5}': 7.142857142857143,
 '{2}': 2.909090909090909,
 '{3}': 4.464285714285714,
 '{4}': 5.96969696969697,
 '{7}': 10.0,
 '{1}{G}{U}': 4.0,
 '{2}{B}{R}': 6.666666666666667,
 '{1}{G}': 3.2658227848101267,
 '{4}{G}{G}': 10.333333333333334,
 '{1}{G}{G}': 3.85,
 '{3}{G}': 5.583333333333333,
 '{3}{G}{G}': 6.782608695652174,
 '{2}{G}{G}': 7.111111111111111,
 '{G}': 2.0,
 '{2}{G}': 3.8620689655172415,
 '{4}{G}': 8.0,
 '{5}{G}{G}': 13.384615384615385,
 '{2}{G}{G}{G}': 13.5,
 '{G}{G}': 3.875,
 '{1}{R}': 3.25,
 '{R}': 2.305084745762712,
 '{3}{R}': 5.34375,
 '{4}{R}{R}': 9.724137931034482,
 '{4}{R}': 6.363636363636363,
 '{2}{R}{R}': 6.184210526315789,
 '{2}{R}': 3.5161290322580645,
 '{1}{R}{R}': 4.8125,
 '{3}{R}{R}': 8.307692307692308,
 '{R}{R}': 4.0,
 '{4}{R}{R}{R}': 12.666666666666666,
 '{5}{R}{R}': 9.6,
 '{2}{R}{R}{R}': 9.5,
 '{B}{B}': 3.260869565217391,
 '{4}{B}': 6.72,
 '{B}': 2.0444444444444443,
 '{3}{B}': 4.835820895522388,
 '{1}{B}': 2.703125,
 '{2}{B}': 3.6777777777777776,
 '{3}{B}{B}': 